### Clustering Analysis
En esta sección se analizarán los clusters obtenidos del notebook 3.0

#### Selección del dataframe con los mejores clusters
Se seleccionará el dataframe con los mejores clusters de acuerdo con las métricas de evaluación de calidad que utilizamos en el notebook anterior.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:
# Loading cleaned dataset
dataset_path = "../data/processed/cleaned_dataset.csv"
df = pd.read_csv(dataset_path)
df = df.drop(columns=["ID", "dias_primera_compra", "n_clicks", "info_perfil"])

In [3]:
# Util functions
def scale_minmax(df, column):
    scaler = MinMaxScaler()
    df[column] = scaler.fit_transform(df[[column]])
def visualize_clusters(df_scaled, df_objective, column):
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(df_scaled)
    plt.figure(figsize=(6, 4))
    plt.scatter(principal_components[:, 0], principal_components[:, 1], c=df_objective[column], cmap='viridis', s=1)
    plt.title('Cluster Visualization (PCA)')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.colorbar(label='Cluster')
    plt.show
def evaluate_clusters(df_scaled, df_objective, column):
    print(f"Silhouette = {silhouette_score(df_scaled, df_objective[column]):.4f}\n"
          f"Davies-Bouldin = {davies_bouldin_score(df_scaled, df_objective[column]):.4f}")
    visualize_clusters(df_scaled, df_objective, column)

In [4]:
# Scaling variables
df_scaled = df.copy()
for i in list(df.columns):
    scale_minmax(df_scaled, i)

In [5]:
# K-Means
df_kmeans = df.copy()
kmeans = KMeans(n_clusters=3).fit(df_scaled.values)
df_kmeans["cluster"] = kmeans.labels_

In [6]:
# DBSCAN
df_dbscan = df.copy()
dbscan = DBSCAN(eps = 0.04, min_samples = 50)
df_dbscan['cluster'] = dbscan.fit_predict(df_scaled)
scaled_db = df_scaled[df_dbscan["cluster"] != -1]
df_dbscan = df_dbscan[df_dbscan["cluster"] != -1]

In [7]:
# Gaussian Mixture
df_gaussi = df.copy()
n_clusters_ = 3  
gaussian = GaussianMixture(
    n_components=n_clusters_, 
    covariance_type='full',  
    init_params='kmeans',    
    max_iter=100, 
    tol=1e-3, 
    random_state=42         
)
df_gaussi["cluster"] = gaussian.fit_predict(df_scaled)

In [8]:
# Agglomerative
df_agglom = df.copy()
sample_size = 26000
random_indexes = np.random.choice(df_scaled.shape[0], sample_size, replace=False)
scaled_ag = df_scaled.iloc[random_indexes]
df_agglom = df_agglom.iloc[random_indexes]
agglomerative = AgglomerativeClustering(n_clusters=3, metric='euclidean', linkage='ward')
df_agglom["cluster"] = agglomerative.fit_predict(scaled_ag)

##### Evaluating clusters

Con el siguiente código:
```python
print("Evaluation of K-Means")
evaluate_clusters(df_scaled, df_kmeans, "cluster")
print("\nEvaluation of DBSCAN")
evaluate_clusters(scaled_db, df_dbscan, "cluster")
print("\nEvaluation of Gaussian Mixture")
evaluate_clusters(df_scaled, df_gaussi, "cluster")
print("\nEvaluation of Agglomerative")
evaluate_clusters(scaled_ag, df_agglom, "cluster")
```
Se obtuvo la siguiente salida:
```python
Evaluation of K-Means
    Silhouette = 0.6382
    Davies-Bouldin = 0.5010

Evaluation of DBSCAN
    Silhouette = 0.6605
    Davies-Bouldin = 0.4660

Evaluation of Gaussian Mixture
    Silhouette = 0.6328
    Davies-Bouldin = 0.5062

Evaluation of Agglomerative
    Silhouette = 0.6346
    Davies-Bouldin = 0.5064
```
Todos estos métodos fueron exhaustivamente probados y para el número de 3 clusters los resultados de las métricas fueron los mejores. Por lo tanto, analizaremos los resultados de los clusters retornados con DBSCAN, el cual fue el que tuvo los mejores scores de las métricas.

##### Analyzing the clusters